# Assignment #2: MNIST Digit Classification

The MNIST dataset is a classic benchmark dataset in machine learning. In this document we will be applying a variety of
dense neural network architectures to the MNIST and assessing their predictive capabilities.

## Loading the Data

We'll start by loading the MNIST dataset from tf.keras:

In [11]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(type(x_train))
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

<class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


We can see that keras splits the data for us into a train/val set. We can also see that they've split the data
**86:14**, which is reasonable for 70 000 training examples. For larger datasets (into millions of examples) the
validation split would be significantly smaller, around **1%**.

We can also see that the inputs are 28 x 28 greyscale pixel images. If we were using a convolutional neural net, then
we wouldn't have to reshape the data. However, dense networks require the data to be in vector format:

In [12]:
from tensorflow.keras.utils import to_categorical

x_train, x_test = [x.reshape((-1, 28 * 28)) / 255. for x in (x_train, x_test)]  # reshape and normalize
y_train, y_test = [to_categorical(y, 10) for y in (y_train, y_test)]  # convert to binary matrix

print(x_train.shape)

(60000, 784)


## Defining a Model

Now that our data has been reshaped, we can define a subclassed keras model for creating our dense networks:

In [13]:
from tensorflow.keras.layers import Dense, Softmax
from tensorflow.keras.models import Model

class DenseNetwork(Model):
    def __init__(self,
                 units_per_layer: list = None,
                 activation: 'str' = 'sigmoid',
                 l2_lambda: float = 0.0,
                 **kwargs):
        super().__init__(dynamic=False, **kwargs)  # dynamic=False ensures we run in graph mode
        self.units_per_layer = units_per_layer or [32]  # len(units_per_layers) defines number of hidden layers
        self.activation = activation
        self.l2_lambda = l2_lambda

        # Define the layers used by this model
        self.dense_layers = [
            Dense(u, activation=self.activation,
                  kernel_regularizer=l2(self.l2_lambda),
                  bias_regularizer=l2(self.l2_lambda))
            for u in self.units_per_layer
        ]
        self.final_layer = Dense(10,  # Ensure we always output a length 10 vector
                                 activation=self.activation,
                                 kernel_regularizer=l2(self.l2_lambda),
                                 bias_regularizer=l2(self.l2_lambda))
        self.softmax = Softmax()  # Activation layer for our output

    def call(self, inputs, training=None, mask=None):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)  # Keras builds the weight matrices from the (.., features) of the input x and the units

        output = self.final_layer(x)  # Ensures we output a (batch, 10) vector
        output = self.softmax(output)  #  probablistic vector; allows for backprop on one-hot targets
        return output

tf.keras makes it incredibly easy to create **customizable** models, allowing us to perform hyperparameter searches
with ease. Because there are no *standard* values for hyperparameters, it's difficult to test how a particular
hyperparameter affects a model without performing a grid search on all possible values of all hyperparameters. This is
incredibly costly, however, so we'll cheat a little bit and *assume* each hyperparamter can be tested independently,
by using the defaults defined earlier in the model class for all other hyperparameters not being tested.

## Architecture Search
We'll begin by testing architectures (number of nodes and layers):

In [14]:
import random

from tensorflow.keras.regularizers import l2, l1
from sklearn.model_selection import ParameterGrid

def get_search_space(s: dict):
    return list(ParameterGrid(s))

# Define a search space of hyperparameters
architecture_search_space = {
    'units_per_layer': [
        [64, 32],  # 3 layers  (Since we have a static output layer in our model)
        [128, 64, 32],  # 4 layers
        [512, 512, 512, 256, 128, 64, 32],  # 8 layers
        [512, 512, 512, 512, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32, 32]  # 16 layers
    ]
}

Neural nets are typically built to be conical in shape, i.e. the first layers have many more units than the final
layers. This is because neurons in earlier layers are responsible for finding very **simple** patterns in data
(of which there are more), like edges, while each consecutive layer is responsible for detecting
increasingly **complex** patterns, like faces (of which there are less and less).

Now that we have our search space of new architectures, we can now create our models and begin testing them:

In [15]:
def search_networks(search_space):
    for i, params in enumerate(get_search_space(search_space)):  # Only going to select 5 model, since 36 will take too long
        print('Model {} => '.format(i + 1) + ''.join([k + ': ' + str(v) for k, v in params.items()]))
        model = DenseNetwork(**params)

        # Use default adam optmizer w/ lr = 0.001
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
        print('\ttrain loss: {:0.3f}, val loss: {:0.3f}, val accuracy: {:0.3f}'.format(
            history.history['loss'][-1],
            history.history['val_loss'][-1],
            history.history['val_accuracy'][-1]
        ))

search_networks(architecture_search_space)

Model 1 => units_per_layer: [64, 32]
	train loss: 1.483, val loss: 1.492, val accuracy: 0.967
Model 2 => units_per_layer: [128, 64, 32]
	train loss: 1.475, val loss: 1.485, val accuracy: 0.971
Model 3 => units_per_layer: [512, 512, 512, 256, 128, 64, 32]
	train loss: 1.851, val loss: 1.848, val accuracy: 0.187
Model 4 => units_per_layer: [512, 512, 512, 512, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32, 32]
	train loss: 2.301, val loss: 2.301, val accuracy: 0.113


We can see that performance peaked at the 4 layer neural network (Model 2), and then got significantly worse as we added
more layers. This is due to the **vanishing gradient** problem. As networks get very deep, the partial derivative
of weights in the early layers are dependent on the partials of all of the weights in the later layers, many of which
are very small. Multiplying many numbers close to zero together gives a number *very* close to zero. What results is
a network incapable of training.

## Defining a Residual Network

We can remedy the vanishing gradient problem by using **residual skip connections**, where every few layers
an input will not simply be the activation of the previous layer, but a *summation* of activations. This is best
diplayed in the code:

In [16]:
from tensorflow.keras.layers import Add


class ResidualNetwork(Model):
    def __init__(self,
                 unit_clusters: list = None,
                 activation: str = 'sigmoid',
                 l2_lambda: float = 0.0,
                 **kwargs):
        super().__init__(dynamic=False, **kwargs)
        # Defaults to 15 hidden layers + 5 skip layers + 1 output layer = 21 layer network
        self.unit_clusters = unit_clusters or [
            [512, 512, 512],  [512, 512, 512], [256, 256, 128], [128, 64, 64], [32, 32, 32]
        ]
        self.shortcut_units = [u[-1] for u in self.unit_clusters]
        self.activation = activation
        self.l2_lambda = l2_lambda

        self.residual_layers = [
            [
                Dense(u,
                      self.activation,
                      kernel_regularizer=l2(self.l2_lambda),
                      bias_regularizer=l2(self.l2_lambda)) for u in cluster
            ] for cluster in self.unit_clusters
        ]
        self.skip_layers = [
            Dense(cluster[-1],
                  self.activation,
                  kernel_regularizer=l2(self.l2_lambda),
                  bias_regularizer=l2(self.l2_lambda)) for cluster in self.unit_clusters
        ]

        self.output_layer = Dense(10)
        self.softmax = Softmax()

    def call(self, inputs, training=None, mask=None):
        x = inputs
        for skip_layer, cluster in zip(self.skip_layers, self.residual_layers):
            x_shortcut = x
            for layer in cluster:
                x = layer(x)
            x_shortcut = skip_layer(x_shortcut)  # Ensure shapes match for summation

            # The residual skip! x is our input for the next set of layers, and it's the summation of x and x_shortcut
            x = Add()([x, x_shortcut])

        output = self.output_layer(x)
        output = self.softmax(output)
        return output

model = ResidualNetwork()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
print('\ttrain loss: {:0.3f}, val loss: {:0.3f}, val accuracy: {:0.3f}'.format(
    history.history['loss'][-1],
    history.history['val_loss'][-1],
    history.history['val_accuracy'][-1]
))

	train loss: 0.040, val loss: 0.111, val accuracy: 0.974


Comparing the residual network to our feed-forward, we've blown the feed-forward network out of the
water, with an accuracy of **~97%** vs a terrible **11%** after only 10 epochs. However, in the process we've
slightly overfit our training set.

To remedy overfitting, we'll apply some L2 regularization to our residual network. Regularization penalizes very
high weight values which arise when the network places too much *emphasise* (or weight) on a particular neuron
activation. However, that neuron may only be very important in the *training* set, and many not generalize
to the validation set. By ensuring that our weights stay around zero, we can ensure no neuron activates too highly,
leaving the others in the dust.

In [17]:
model = ResidualNetwork(l2_lambda=0.01)  # Suggested value of lambda by keras
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
print('\ttrain loss: {:0.3f}, val loss: {:0.3f}, val accuracy: {:0.3f}'.format(
    history.history['loss'][-1],
    history.history['val_loss'][-1],
    history.history['val_accuracy'][-1]
))


	train loss: 2.310, val loss: 2.312, val accuracy: 0.103


Well. We're no longer overfitting our data, however we've destroyed our results in the process. Since the network
without regularization isn't exhibiting too much overfitting, we can probably get away without it, or at least
use a very, very small lambda value.

## Activation Search

Next we can test various activation functions. We'll do this with our original feed-forward network, since our
residual network is computationally intensive to run:

In [18]:
activation_search_space = {
    'activation': ['linear', 'sigmoid', 'tanh', 'relu', 'selu']  # keras identifies activation functions by string
}
search_networks(activation_search_space)

Model 1 => activation: linear
	train loss: 0.256, val loss: 0.279, val accuracy: 0.922
Model 2 => activation: sigmoid
	train loss: 1.509, val loss: 1.515, val accuracy: 0.945
Model 3 => activation: tanh
	train loss: 0.861, val loss: 0.879, val accuracy: 0.959
Model 4 => activation: relu
	train loss: 0.071, val loss: 0.107, val accuracy: 0.968
Model 5 => activation: selu
	train loss: 0.074, val loss: 0.118, val accuracy: 0.967


Surprisingly, a linear activation function acheives an accuracy of **92%**, suggesting the MNIST is largely linearly
separable. Of the activations tests, the **selu** was the clear winner, with an accuracy of **96.9%**. The selu is an
interesting activation:

![selu](./selu.png)

We can see that a selu is very similar to a relu activation function, but instead of a hinge at x = 0, it
gradually increases to a linear trend. This non-linearity around 0 supposedly allows the network to "self normalize"
itself, however the exact mathematical rigor of this claim is
[up for debate](https://media.nips.cc/nipsbooks/nipspapers/paper_files/nips30/reviews/617.html).

## Conclusion

We've tested various architectures, regularization techniques and activation functions separately, let's now
combine the best bits to see if we can create a model with the highest accuracy yet:

In [19]:
# We'll use a residual network with 16 layers

model = ResidualNetwork(
    unit_clusters=None,  # Use the default 16 layer architecture defined in the model
    activation='selu',  # Self normalizing linear unit
    l2_lambda=0.000001  # lambda ~0 so we don't nuke our performance from orbit
)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 13s 221us/sample - loss: 0.4167 - accuracy: 0.8864 - val_loss: 0.2347 - val_accuracy: 0.9358

And there we have it. An accuracy of **~98%** in only 10 epochs. We're stilling overfitting, but our 
validation results are respectable. If we wanted to really push predictive capabilities on
the MNIST, we should really be using a convolutional neural network; architectures designed for image data. Conv nets
[easily achieve](https://en.wikipedia.org/wiki/MNIST_database) upwards of **99.8%** accuracy on the MNIST, however
an accuracy of **98%** from a dense network is quite respectable. Also, 15 epochs is an incredibly short time
to train a network. Large networks are often trained for thousands, if not tens of thousands of epochs before
their results are reported. From our values, it's obvious that our model has **yet to converge**. However, since the
residual network is intensive to run (especially on a CPU), 15 epochs is more than enough to show
predictive capabilites.